In [8]:
# 1) Make sure nbformat is installed
# pip install nbformat

import os
from pathlib import Path
import nbformat as nbf

# --- Choose a writable project root on your machine ---
project_root = Path.home() / "Desktop" / "somali-music-feature-explorer"   # <-- adjust if you want
nb_dir = project_root / "notebooks"

# 2) Create required directories
(nb_dir).mkdir(parents=True, exist_ok=True)

# 3) Build the notebook
nb = nbf.v4.new_notebook()
cells = []

cells.append(nbf.v4.new_markdown_cell(
    "# 02 – Compare Two Songs: MFCC Similarity + PCA\n"
    "Drop two audio files in `../data/` (e.g., `song_a.wav`, `song_b.wav`).\n\n"
    "**What this notebook does**\n"
    "- Loads two tracks\n"
    "- Extracts MFCCs and pitch (PYIN)\n"
    "- Computes cosine similarity of MFCC statistics\n"
    "- (Optional) Dynamic Time Warping (DTW) similarity on MFCC sequences\n"
    "- Runs PCA to visualize both tracks in a 2D feature space\n\n"
    "**Tip:** Keep clips short (15–30s) for quick iteration."
))

cells.append(nbf.v4.new_code_cell(
"""# !pip install librosa matplotlib numpy scikit-learn --quiet
import os
import numpy as np
import librosa, librosa.display
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from numpy.linalg import norm

# ---- Configure your two files here ----
file_a = "../data/song_a.wav"
file_b = "../data/song_b.wav"

assert os.path.exists(file_a), f"Missing file: {file_a}"
assert os.path.exists(file_b), f"Missing file: {file_b}"

SR = 22050  # analysis sample rate
y_a, sr_a = librosa.load(file_a, sr=SR, mono=True)
y_b, sr_b = librosa.load(file_b, sr=SR, mono=True)

print(f"A: {file_a} | len={len(y_a)/SR:.2f}s  | sr={SR}")
print(f"B: {file_b} | len={len(y_b)/SR:.2f}s  | sr={SR}")"""
))

cells.append(nbf.v4.new_code_cell(
"""# ---- MFCC features ----
n_mfcc = 20
mfcc_a = librosa.feature.mfcc(y=y_a, sr=SR, n_mfcc=n_mfcc)
mfcc_b = librosa.feature.mfcc(y=y_b, sr=SR, n_mfcc=n_mfcc)

# Summaries for cosine similarity (per-track statistics)
mfcc_a_mean = mfcc_a.mean(axis=1)
mfcc_a_std  = mfcc_a.std(axis=1)
mfcc_b_mean = mfcc_b.mean(axis=1)
mfcc_b_std  = mfcc_b.std(axis=1)

# Two summary vectors: mean and [mean|std] concatenated
vec_a_mean = mfcc_a_mean
vec_b_mean = mfcc_b_mean

vec_a_meanstd = np.concatenate([mfcc_a_mean, mfcc_a_std])
vec_b_meanstd = np.concatenate([mfcc_b_mean, mfcc_b_std])

def cosine_sim(u, v):
    return float(np.dot(u, v) / (norm(u) * norm(v) + 1e-8))

print("Cosine similarity (means only):      ", round(cosine_sim(vec_a_mean, vec_b_mean), 4))
print("Cosine similarity (means + std dev): ", round(cosine_sim(vec_a_meanstd, vec_b_meanstd), 4))"""
))

cells.append(nbf.v4.new_code_cell(
"""# ---- (Optional) DTW distance on MFCC sequences ----
from librosa.sequence import dtw

def zscore(m):
    m2 = (m - m.mean(axis=1, keepdims=True)) / (m.std(axis=1, keepdims=True) + 1e-8)
    return m2

Za = zscore(mfcc_a)
Zb = zscore(mfcc_b)

D = 1 - np.dot(Za.T, Zb) / ((norm(Za.T, axis=1, keepdims=True) * norm(Zb, axis=0, keepdims=True)) + 1e-8)

wp, cost = dtw(D=D)
dtw_dist = cost[-1, -1] / (len(wp) + 1e-8)
print("DTW normalized distance (MFCC): ", round(float(dtw_dist), 4))"""
))

cells.append(nbf.v4.new_code_cell(
"""# ---- Pitch (PYIN) overview (optional plots) ----
f0_a, vf_a, _ = librosa.pyin(y_a, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'), sr=SR)
f0_b, vf_b, _ = librosa.pyin(y_b, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'), sr=SR)
t_a = librosa.times_like(f0_a, sr=SR)
t_b = librosa.times_like(f0_b, sr=SR)

plt.figure(figsize=(12,4))
plt.plot(t_a, f0_a, label="A: f0", linewidth=1)
plt.plot(t_b, f0_b, label="B: f0", linewidth=1, alpha=0.8)
plt.xlabel("Time (s)"); plt.ylabel("Pitch (Hz)")
plt.title("Pitch (F0) – PYIN")
plt.legend(); plt.tight_layout(); plt.show()

print("Mean F0 A:", np.nanmean(f0_a))
print("Mean F0 B:", np.nanmean(f0_b))"""
))

cells.append(nbf.v4.new_code_cell(
"""# ---- PCA visualization on pooled MFCC frames ----
n_frames = min(mfcc_a.shape[1], mfcc_b.shape[1])
Xa = mfcc_a[:, :n_frames].T
Xb = mfcc_b[:, :n_frames].T

X = np.vstack([Xa, Xb])
labels = np.array(["A"]*n_frames + ["B"]*n_frames)

pca = PCA(n_components=2, random_state=0)
X2 = pca.fit_transform(X)

plt.figure(figsize=(7,6))
mask_a = labels == "A"
mask_b = ~mask_a
plt.scatter(X2[mask_a,0], X2[mask_a,1], label="A", alpha=0.6)
plt.scatter(X2[mask_b,0], X2[mask_b,1], label="B", alpha=0.6)
plt.xlabel("PC1"); plt.ylabel("PC2"); plt.title("PCA of MFCC Frames")
plt.legend(); plt.tight_layout(); plt.show()

print("Explained variance ratio:", pca.explained_variance_ratio_)"""
))

nb["cells"] = cells

nb_path = nb_dir / "02_compare_songs.ipynb"
with open(nb_path, "w", encoding="utf-8") as f:
    nbf.write(nb, f)

print("Notebook created at:", nb_path)


Notebook created at: /Users/khalidibrahim/Desktop/somali-music-feature-explorer/notebooks/02_compare_songs.ipynb
